# La familia *Apply*



Profesor: Rodrigo Manzanas (rodrigo.manzanas@unican.es)

## Objetivos de la sesión

Al finalizar esta sesión, el estudiante será capaz de:
- Comprender la utilidad de la familia *Apply* para evitar bucles explícitos.
- Identificar cuándo usar `apply`, `lapply`, `sapply`, `mapply`, `tapply`, `rapply`, `vapply`.
- Comparar eficiencia y legibilidad entre bucles y funciones *Apply*.
- Reconocer buenas prácticas y errores comunes al usar funciones *Apply*.

## Contexto práctico

Las estructuras de control permiten automatizar tareas, tomar decisiones y procesar datos de manera eficiente. Por ejemplo, son esenciales para limpiar datos, realizar cálculos repetitivos o aplicar reglas de negocio en análisis de datos. En este sentido, las estructuras de control son el esqueleto del lenguage de la programación. Entre las estructuras de control encontramos los bucles o las sentencias condicionales. De este modo, el objetivo de estas sesiones vamos a ver las estructuras de control básicas de la programación en R. 

PARTE 1:
- if-else
- for
- while
- repeat/break

PARTE 2:
- apply/sapply/lapply/tapply
- programación paralelizada

## Tabla resumen de la familia *Apply*

| Función   | Estructura sobre la que opera | Output         | Uso típico                      |
|-----------|-------------------------------|----------------|---------------------------------|
| apply     | matriz/array                  | vector/matriz  | Operar por filas/columnas       |
| lapply    | lista/data.frame              | lista          | Iterar sobre listas             |
| sapply    | lista/data.frame              | vector/matriz  | Igual que lapply, simplifica    |
| mapply    | varias listas/vectores        | lista/vector   | Iterar sobre varios objetos     |
| tapply    | vector + factor               | vector/array   | Agrupar y aplicar función       |
| rapply    | lista (recursiva)             | lista/vector   | Recorrer listas anidadas        |
| vapply    | lista/data.frame              | tipo fijo      | Igual que sapply, tipo seguro   |

## Ejemplo comparativo: bucle for vs. apply

In [1]:
# Suma de filas de una matriz con bucle for
set.seed(1)
mat <- matrix(sample(1:100, 20), nrow=5)
suma_filas_for <- numeric(nrow(mat))
for (i in 1:nrow(mat)) {
  suma_filas_for[i] <- sum(mat[i,])
}
suma_filas_for

# Suma de filas con apply
suma_filas_apply <- apply(mat, 1, sum)
suma_filas_apply

[1] 269 153 174 250 242

[1] 269 153 174 250 242

### Introducción

`apply()` y sus variantes (`lapply()`, `sapply()`, `mapply()`...) son funciones para manipular matrices, arrays, listas y dataframes de manera repetitiva, permitiendo evitar el uso de ciclos `for` y `while`. 

Comenzamos con un ejemplo sencillo, supongamos que queremos poner en minúsculas las letras de todos los nombres de un vector. La función de `R` que lo hace es `tolower()`. La forma habitual y primera aproximación para hacerlo es utilizar un ciclo `for`, como sigue:

In [2]:
nombres <- c("Paco","Álvaro","María", "Alex")
for (i in 1:length(nombres)){
    nombres[i] <- tolower(nombres[i])
}
nombres

[1] "paco"   "álvaro" "maría"  "alex"

Pero podemos escribirlo de manera más concisa usando la función `lapply()`:

In [3]:
nombres <- c("Paco","Álvaro", "María", "Alex")
nombres <- lapply(nombres, tolower)
nombres
class(nombres)

[[1]]
[1] "paco"

[[2]]
[1] "álvaro"

[[3]]
[1] "maría"

[[4]]
[1] "alex"

[1] "list"

Nota: Hay que tener en cuenta que muchas funciones de `R` están *vectorizadas*, lo que significa que pueden aplicarse directamente sobre vectores:

In [4]:
tolower(nombres)

[1] "paco"   "álvaro" "maría"  "alex"

Por supuesto, para cualquier función no vectorizada, o para navegar sobre tipos de datos más complejos, la familia *Apply* será muy útil.

En la siguiente sección veremos en detalle las diferencias entre las diferentes funciones de la familia; esencialmente cada una de ellas está indicada para operar sobre un tipo de estructura distinta. Nos centraremos en `apply()`, pensada para recorrer matrices sobre sus dimensiones; `lapply()`, pensada para recorrer listas y vectores; `sapply()`, que equivale a `lapply()` pero trata de simplificar en un array el resultado, y `mapply()`, que permite iterar sobre varias listas o vectores a la vez.

## `apply()`

Esta función opera sobre arrays multidimensionales, en particular matrices. Sus tres argumentos de entrada principales son:

- `X`: Array sobre el que se desea operar.
- `MARGIN`: Dimensión sobre la que se desea aplicar la función indicada en `FUN`. Escribiendo `MARGIN = 1` aplicaremos la función a las filas de la matriz y escribiendo `MARGIN = 2` sobre las columnas. Si `X` tiene más de dos dimensiones `MARGIN = 3` opera sobre la tercera dimensión, etc...
- `FUN`: Función a aplicar.


Vamos a trabajar con la matriz `A`:

In [5]:
set.seed(1) # resultados reproducibles
A <- replicate(expr = sample(1:15, size = 6), n = 4) # replicate() evalua la expresión expr n veces
A

9,11,5,9
4,14,15,5
7,2,10,14
1,15,6,15
2,3,13,12
13,1,7,13


**Ejemplo:** Cálculo del máximo de cada fila de la matriz:

In [6]:
apply(X = A, MARGIN = 1, FUN = max)

[1] 11 15 14 15 13 13

**Ejemplo:** Cálculo del mínimo de cada columna de la matriz:

In [7]:
apply(X = A, MARGIN = 2, FUN = min)

[1] 1 1 5 5

**Ejemplo:** Calcular el mímimo de cada columna de la matriz `A`, a la que hemos introducido `NA`:

In [8]:
# Introducimos NAs
A[1,2] <- NA
A[3,2] <- NA
A[4,3] <- NA

print(A)
apply(X = A, MARGIN = 2, FUN = min)

     [,1] [,2] [,3] [,4]
[1,]    9   NA    5    9
[2,]    4   14   15    5
[3,]    7   NA   10   14
[4,]    1   15   NA   15
[5,]    2    3   13   12
[6,]   13    1    7   13


[1]  1 NA NA  5

Como vemos, al haber `NA` en la matriz, el mínimo de alguna columna es `NA` también. 
`apply()` permite pasar argumentos adicionales a la función especificada en `FUN`, por lo que, si queremos evitar este comportamiento podríamos pasarle el argumento `na.rm = TRUE` a la función `min()`:

In [9]:
apply(X = A, MARGIN = 2, FUN = min, na.rm = TRUE)

[1] 1 1 5 5

También podemos usar `apply()` sobre arrays de más dimensiones, por ejemplo:

**Ejemplo:** Cálculo de la media sobre la tercera dimensión en un array tridimensional

In [10]:
A3 <- replicate(replicate(expr = sample(1:15, size = 5), n = 3), n = 10) # A3 es un array de dimensiones 5x3x10
print(A3[,,10]) # ejemplo
apply(A3, MARGIN = 3, FUN = mean)  # media sobre la tercera dimensión

     [,1] [,2] [,3]
[1,]    1   14    9
[2,]    2    3    2
[3,]   11   13   12
[4,]    3   11   10
[5,]   12    8    8


[1] 8.533333 8.400000 8.400000 7.400000 9.200000 7.200000 7.466667 7.333333
 [9] 7.400000 7.933333

Hay que tener en cuenta que podemos escribir nuestra propia función dentro de la función `apply()`, lo que será muy útil para efectuar operaciones específicas. El ejemplo anterior equivale a:

In [11]:
apply(A3, MARGIN = 3, FUN = function(x) {return(mean(x))})

[1] 8.533333 8.400000 8.400000 7.400000 9.200000 7.200000 7.466667 7.333333
 [9] 7.400000 7.933333

**Ejercicio:** Obtén la media por filas de la matriz `A`, ignorando los `NA`:

**Nota**: En `R` existen las funciones `rowMeans()`, `rowSums()`, `colMeans()`, `colSums()`. Éstas están ya precompiladas en `C` y son mucho más rápidas que sus equivalentes utilizando bien `apply()` o ciclos `for`.

**Ejercicio:** En la matriz `A`, queremos saber cuántos `NA` hay en cada columna (recordar funciones `sum()` e `is.na()`):

In [12]:
print(A)


     [,1] [,2] [,3] [,4]
[1,]    9   NA    5    9
[2,]    4   14   15    5
[3,]    7   NA   10   14
[4,]    1   15   NA   15
[5,]    2    3   13   12
[6,]   13    1    7   13


**Ejercicio:** Obtén la matriz resultado de sumar a cada columna de la matriz `A` el vector `y`:

In [13]:
y <- seq(1,6)


`apply()` puede utilizarse también sobre un `data.frame`, tratándolo como si fuera una matriz.

**Ejercicio**: Del `data.frame` `alumnos`, verifica si hay alguien que se llame "jose" e identifica la carrera que ha estudiado. Apóyate en la función `is.element`

In [14]:
alumnos <-
data.frame(fisica = c("juan", "pablo", "maria", "jose"),
           matematicas = c("joaquin", "maialen", "carlos", NA),
           biologia = c("ana", "daniel", "markel", "adriana"))
print(alumnos)


  fisica matematicas biologia
1   juan     joaquin      ana
2  pablo     maialen   daniel
3  maria      carlos   markel
4   jose        <NA>  adriana


## `lapply()`

`lapply()` es la función indicada para trabajar con listas, aunque también permite iterar sobre vectores y sobre `data.frames`. Los argumentos de entrada de`lapply` son:

- `X`: Lista sobre la que se desea iterar.
- `FUN`: Función a aplicar.
- (`...`): Argumentos adicionales para `FUN`.

**Ejercicio:** De la lista siguiente obtén la media de cada vector

In [15]:
lista.vec <- list(c(9,10,5,6), 
                    c(9,9,8,4),
                    c(4,3,6,6))
print(lista.vec)
lapply(lista.vec, mean)

[[1]]
[1]  9 10  5  6

[[2]]
[1] 9 9 8 4

[[3]]
[1] 4 3 6 6



[[1]]
[1] 7.5

[[2]]
[1] 7.5

[[3]]
[1] 4.75

**Ejercicio:** De la lista de matrices siguientes, obtén las medias por columnas de cada matriz. Utiliza para ello `apply` dentro de `lapply`

In [16]:
lista.mat <- replicate(replicate(expr = sample(1:15, size = 6), n = 3), n = 4, simplify = F) # simplify = FALSE no simplifica en un array
print(lista.mat)



[[1]]
     [,1] [,2] [,3]
[1,]    9   11    9
[2,]    6   12   13
[3,]    1    5    5
[4,]    4    8    1
[5,]    5    4   14
[6,]   15    1    4

[[2]]
     [,1] [,2] [,3]
[1,]   10    5   14
[2,]   14    6   15
[3,]   15   14    4
[4,]    9    2   10
[5,]    8   12    8
[6,]    5    8    5

[[3]]
     [,1] [,2] [,3]
[1,]    5    1   15
[2,]    8   10    9
[3,]   14    4    6
[4,]    7    9   13
[5,]    4   12    4
[6,]   11   11    3

[[4]]
     [,1] [,2] [,3]
[1,]   13    9   14
[2,]    3   14    4
[3,]   15    5   13
[4,]    9   12   10
[5,]   12    7    8
[6,]    7    4    1



Debe tenerse en cuenta que: 

1. El objeto devuelto por `lapply` siempre es una lista, incluso aunque el objeto sobre el que iteramos no lo sea. Lo podemos ver con nuestro ejemplo inicial:

In [17]:
nombres <- c("Paco","Álvaro","María", "Alex")
is.list(lapply(nombres, tolower))

[1] TRUE

2. Si se aplica `lapply()` a una matriz, ésta es tratada como un vector, es decir, `FUN` opera elemento a elemento.

**Ejemplo:**

In [18]:
print(A)
lapply(A, mean)

     [,1] [,2] [,3] [,4]
[1,]    9   NA    5    9
[2,]    4   14   15    5
[3,]    7   NA   10   14
[4,]    1   15   NA   15
[5,]    2    3   13   12
[6,]   13    1    7   13


[[1]]
[1] 9

[[2]]
[1] 4

[[3]]
[1] 7

[[4]]
[1] 1

[[5]]
[1] 2

[[6]]
[1] 13

[[7]]
[1] NA

[[8]]
[1] 14

[[9]]
[1] NA

[[10]]
[1] 15

[[11]]
[1] 3

[[12]]
[1] 1

[[13]]
[1] 5

[[14]]
[1] 15

[[15]]
[1] 10

[[16]]
[1] NA

[[17]]
[1] 13

[[18]]
[1] 7

[[19]]
[1] 9

[[20]]
[1] 5

[[21]]
[1] 14

[[22]]
[1] 15

[[23]]
[1] 12

[[24]]
[1] 13

3. Si se aplica `lapply` sobre un `data.frame`, éste se itera por columnas.

**Ejercicio:** Del `data.frame` `alumnos`, verifica si hay alguien que se llame "jose" e identifica la carrera que ha estudiado, esta vez utilizando `lapply()`:

In [19]:
alumnos <-
data.frame(fisica = c("juan", "pablo", "maria", "jose"),
           matematicas = c("joaquin", "maialen", "carlos", NA),
           biologia = c("ana", "daniel", "markel", "adriana"))
print(alumnos)


  fisica matematicas biologia
1   juan     joaquin      ana
2  pablo     maialen   daniel
3  maria      carlos   markel
4   jose        <NA>  adriana


## `sapply()`

La función `sapply()` es idéntica a `lapply()`, pero trata de simplificar el resultado a un vector, matriz o array.  
**Ejemplos:**

In [20]:
nombres <- c("Paco","Álvaro","María", "Alex")
is.vector(sapply(nombres, tolower))  # vector de strings

[1] TRUE

In [21]:
alumnos <-
data.frame(fisica = c("juan", "pablo", "maria", "jose"),
           matematicas = c("joaquin", "maialen", "carlos", NA),
           biologia = c("ana", "daniel", "markel", "adriana"))
is.vector(sapply(alumnos, is.element, el = "jose"))  # vector lógico (valores booleanos)

[1] TRUE

`sapply()` tiene dos argumentos adicionales respecto a `lapply()`:
- `simplify` Si se especifica `simplify = FALSE`, entonces es idéntico a `lapply()`.
- `USE.NAMES` Si se especifica `USE.NAMES = TRUE` (por defecto), y `X` es de tipo `character`, se usan estos como nombres para el output.

In [22]:
nombres <- c("Paco","Álvaro","María", "Alex")
sapply(nombres, tolower)  # vector de strings
sapply(nombres, tolower, USE.NAMES = FALSE)  # vector de strings

Paco   Álvaro    María     Alex 
  "paco" "álvaro"  "maría"   "alex"

[1] "paco"   "álvaro" "maría"  "alex"

## `mapply()`

Hasta ahora hemos visto cómo operar sobre un *único* objeto (por ejemplo una lista o una matriz). `mapply()` (*multivariate apply*) permite operar sobre más de un objeto a la vez.

Sus argumentos de entrada son:
- `FUN`: Función a aplicar.
- (`...`): Argumentos sobre los que vectorizar.
- `MoreArgs`: Lista de argumentos adicionales (no vectorizados).
- `SIMPLIFY`: Si se marca como `TRUE`, se intentará *simplificar* el resultado a un vector, matriz o array multidimensional.


**Ejemplo**: En este caso tenemos dos listas de vectores, y queremos sumar vector a vector. Podemos hacerlo con `mapply()`:

In [23]:
lista.vec1 <- list(c(1,1,10,20), c(-1,12,122,44), c(34,12,65,23))
lista.vec2 <- list(c(100,200,100,210), c(400,120,1220,104), c(-530,-320,630,-650))
mapply(FUN = function(x,y) x+y, lista.vec1, lista.vec2)

101,399,-496
201,132,-308
110,1342,695
230,148,-627


Por defecto `mapply()` simplifica el resultado en una matriz. Podemos evitarlo si especificamos `SIMPLIFY = FALSE`, lo que nos devolverá como resultado una lista:

In [24]:
mapply(FUN = function(x,y) x+y, lista.vec1, lista.vec2, SIMPLIFY = FALSE)

[[1]]
[1] 101 201 110 230

[[2]]
[1]  399  132 1342  148

[[3]]
[1] -496 -308  695 -627

La manera de asignar argumentos adicionales es mediante el argumento `MoreArgs` (atención, `MoreArgs`debe ser una lista). 

**Ejercicio:** ¿Qué crees que hace el siguiente código?

In [25]:
#mapply(FUN = function(x,y, absoluto = FALSE) {
#                if (absoluto) return(abs(x+y))
#                else return(x+y)
#             },
#       lista.vec1, lista.vec2, SIMPLIFY = FALSE, MoreArgs = list(absoluto = TRUE)
#     )

**Ejercicio:** De la lista `mi.lista.1`, obtén el elemento $5$ del primer vector, el elemento $4$ del segundo vector y el elemento $2$ del tercer vector:

In [26]:
mi.lista.1 <- replicate(sample(1:10, size = 5, replace = T), n = 3, simplify = F)  # lista con 3 elementos, donde cada elemento es un vector de 5 componentes
print(mi.lista.1)


[[1]]
[1] 4 2 5 2 4

[[2]]
[1] 3 6 9 7 5

[[3]]
[1]  5  1  1 10  1



**Ejercicio:** Las listas `mi.lista.1` y `mi.lista.2` contienen el mismo número de vectores. Obtén, para cada par de vectores, el valor máximo:

In [27]:
mi.lista.2 <- replicate(sample(1:50, size = 5, replace = T), n = 3, simplify = F)  # lista con 3 elementos, donde cada elemento es un vector de 5 componentes
print(mi.lista.1)
print(mi.lista.2)
  

[[1]]
[1] 4 2 5 2 4

[[2]]
[1] 3 6 9 7 5

[[3]]
[1]  5  1  1 10  1

[[1]]
[1] 16 49 27 39 38

[[2]]
[1] 16 34 29 45 35

[[3]]
[1] 10  6 39 49 33



**Ejercicio:** Repite el ejercio anterior. Esta vez los vectores pueden tener `NA` y queremos ignorarlos:

In [28]:
mi.lista.1[[1]][3] <- NA
mi.lista.2[[3]][4] <- NA
print(mi.lista.1)
print(mi.lista.2)

mapply(FUN = max, mi.lista.1, mi.lista.2, MoreArgs = list(na.rm = TRUE))  

[[1]]
[1]  4  2 NA  2  4

[[2]]
[1] 3 6 9 7 5

[[3]]
[1]  5  1  1 10  1

[[1]]
[1] 16 49 27 39 38

[[2]]
[1] 16 34 29 45 35

[[3]]
[1] 10  6 39 NA 33



[1] 49 45 39

**Ejercicio:** En el `data.frame` `alumnos`, verifica usando `mapply()` si alguien que se llame "jose" ha estudiado física, alguien que se llame "maialen" ha estudiado matemáticas y alguien que se llame "raquel" ha estudiado biología.

In [29]:
print(alumnos)


  fisica matematicas biologia
1   juan     joaquin      ana
2  pablo     maialen   daniel
3  maria      carlos   markel
4   jose        <NA>  adriana


**Ejercicio:** Para cada par de vectores de las listas `lista.vec1` y `lista.vec2`, obtén el elemento que es mayor en valor absoluto. Utiliza para ello `mapply` y una función `max.val.abs` que definida por tí.

In [30]:
lista.vec1 <- replicate(sample(-20:20, size = 5, replace = T), n = 3, simplify = F)  # lista con 3 elementos, donde cada elemento es un vector de 5 componentes
lista.vec2 <- replicate(sample(-10:10, size = 5, replace = T), n = 3, simplify = F)  # lista con 3 elementos, donde cada elemento es un vector de 5 componentes
print(lista.vec1)
print(lista.vec2)



[[1]]
[1]  19  13 -17   4  17

[[2]]
[1]  -4   3 -12   9 -12

[[3]]
[1] -10   1  11  19 -18

[[1]]
[1]   8   8 -10   9  -7

[[2]]
[1]   2 -10  -3  10  -6

[[3]]
[1] -1  1 -8 10 -8



## Otras funciones: `rapply()`, `tapply()` y `vapply()`

Existen más funciones en la familia *Apply* diseñadas para tareas más específicas. En esta sección veremos `rapply()`, `tapply()` y `vapply()` someramente a través de ejemplos.

### `rapply()`

Esta función está pensada para operar **recursivamente** sobre listas. Es especialmente útil cuando nos encontramos con listas mal formateadas que contienen distintos tipos de objetos.

  **Ejemplo:** La siguiente lista contiene números y listas de números. Supongamos que queremos asignar un `NA` a todos los elementos de la lista que sean negativos. Observa lo que ocurre si se utiliza `lapply()`:

In [31]:
lista.e <- list(1, 2, -1, list(10,12,-1), 4, -4, list(1,1), list(-1,-1,-2,5), 3, 6, 1, -1)
print(lista.e)
lapply(lista.e, function(x){
                    if(x<0) return(NA)
                    else return(x)
    } )

[[1]]
[1] 1

[[2]]
[1] 2

[[3]]
[1] -1

[[4]]
[[4]][[1]]
[1] 10

[[4]][[2]]
[1] 12

[[4]][[3]]
[1] -1


[[5]]
[1] 4

[[6]]
[1] -4

[[7]]
[[7]][[1]]
[1] 1

[[7]][[2]]
[1] 1


[[8]]
[[8]][[1]]
[1] -1

[[8]][[2]]
[1] -1

[[8]][[3]]
[1] -2

[[8]][[4]]
[1] 5


[[9]]
[1] 3

[[10]]
[1] 6

[[11]]
[1] 1

[[12]]
[1] -1



ERROR: Error in if (x < 0) return(NA) else return(x): the condition has length > 1


`rapply()` permite operar de manera recursiva sobre la lista sin preocuparnos del mal formato:

In [ ]:
rapply(lista.e, function(x){
                    if(x<0) return(NA)
                    else return(x)
                })

[1]  1  2 NA 10 12 NA  4 NA  1  1 NA NA NA  5  3  6  1 NA

No sólo eso, también permite especificar el tipo de argumento sobre el que se quiere operar utilizando el argumento `clases`. 

  **Ejercicio**: Utiliza `rapply()` para elevar al cuadrado todos los elementos númericos de la siguiente lista:

In [ ]:
lista.e2 <- list(2, 1, 2, NA, 12, "err", list("err", 1, 3), 5, NA, list(), NULL, 2)



### `tapply()`

`tapply()` está indicada cuando hay que aplicar una función a una variable, dividiendo ésta en grupos según dicta otra variable. La variable sobre la que iterar es, como viene siendo habitual, `X`, y el argumento por el que queremos agrupar es `INDEX`.

**Ejercicio**: Utiliza `tapply()` para calcular, en el dataset `iris`, la media de la variable $Sepal.Length$ para cada especie:

### `vapply()`

Esta función es similar a `sapply()`, con la diferencia de que es posible controlar la clase del output, razón por la cual `vapply()` es considerada más robusta que `sapply()`, ya que evita resultados inesperados a la hora de trabajar con estructuras complejas. No sólo eso, también puede ser ligeramente más eficiente, puesto que no tiene que decidir la estructura de salida, ya que la impone el usuario. Veámoslo con un par de **ejemplos:**

Queremos, de `lista1`, devolver `TRUE` si la sublista contiene alguna `FALSE` si no. Programamos nuestra función `busca.e`:

In [ ]:
lista1 <- list( c("a","e","i","o","u"), c("m","n","l","r"), c("f","e","r","m","e"))
busca.e <- function(x) x=="e"

Buscamos un vector de tres elementos que contenga `TRUE` si la lista contenía alguna `"e"` y `FALSE` si no.

In [ ]:
sapply(lista1, busca.e)

[[1]]
[1] FALSE  TRUE FALSE FALSE FALSE

[[2]]
[1] FALSE FALSE FALSE FALSE

[[3]]
[1] FALSE  TRUE FALSE FALSE  TRUE

Sin embargo, como `busca.e` no la hemos hecho del todo bien, obtenemos una lista que no es lo que esperamos. Es más robusto controlar la estructura pidiendo que nos devuelva específicamente un objeto de tipo `logical` de longitud $1$, lo que puede hacerse con `vapply()`:

In [ ]:
vapply(lista1, busca.e, logical(1))

ERROR: Error in vapply(lista1, busca.e, logical(1)): Los valores deben ser de longitud 1, 
pero el resultado FUN(X [[1]]) es la longitud 5 


`vapply()` comprueba el output y nos avisa de que estamos intentando devolver resultados de longitud $5$ en lugar de resultados de longitud $1$ (*`values must be length 1`*). 

## Buenas prácticas y errores comunes

- Prefiere funciones vectorizadas (`rowMeans`, `colSums`, etc.) cuando estén disponibles.
- Usa `vapply` si necesitas asegurar el tipo y longitud del resultado.
- Recuerda que `lapply` siempre devuelve una lista, aunque el input sea un vector.
- Si la función aplicada puede devolver `NA`, usa argumentos como `na.rm=TRUE`.

## Recursos adicionales

- [Control Structures - R Documentation](https://stat.ethz.ch/R-manual/R-devel/library/base/html/apply.html)
- [R for Data Science: Iteration](https://r4ds.had.co.nz/iteration.html)

## Ejercicios de autoevaluación

- Usa `apply` para calcular la desviación estándar de cada columna de una matriz.
- Usa `lapply` para transformar una lista de data.frames, extrayendo una columna específica.
- Utiliza `tapply` para agrupar y sumar valores por factor en un vector.
- Modifica un ejemplo para que utilice `vapply` en vez de `sapply`.

### Nota final

Utilizar las funciones de la familia *Apply* permite escribir código en `R` de manera concisa y eficiente. Sin embargo, estas funciones no son *per se* más rápidas que un ciclo `for` bien definido. En ese sentido, la razón por la que habitualmente las funciones de la familia *Apply* resultan ser más rápidas es porque éstas reservan espacio en memoria de antemano de manera implícita. Si se hace esto manualmente y se programa correctamente el ciclo `for`, puede comprobarse que los tiempos de ejecución son similares.

Nótese por último que si existe una función vectorizada para la operación que se quisiera llevar a cabo, sea con `apply()` o un ciclo, conviene usar dicha función. Por ejemplo, calcular la media por columnas de una matriz puede hacerse con un ciclo y mediante el comando `apply(foo, 2, mean)`, pero es más eficiente si se utiliza la función de `colMeans(foo)`.

Por último, las funciones de la familia *Apply* abren la puerta a **paralelizar** de manera inmediata.

# Paralelizando con la librería *parallel*

Cualquier ciclo que programemos se ejecuta de manera secuencial, lo que implica que se está usando un solo procesador. Paralelizar implica subdividir la tarea en subtareas para que sean procesadas por los diferentes núcleos/nodos (o procesadores, si nuestra máquina tiene más de uno).

Las funciones y metodología que hemos visto hasta ahora permiten paralelizar procesos de manera muy sencilla. Precisamente porque se está repitiendo una tarea múltiples veces de manera independiente, todas ellas son de naturaleza paralelizable. Para hacerlo, basta tener en cuenta que, a excepción de `mapply()`, las funciones que hemos visto tienen su equivalente en la librería `parallel`: `parApply()` `parLapply()`, `parSapply()`.  

Funcionan exactamente igual, pero requieren como primer argumento un objeto de tipo *cluster* de `parallel`, que podemos iniciar con la función `makeCluster()`.

En Mac/Linux la sintaxis básica para iniciar este objeto es `makeCluster(numero_nodos, type = "FORK")`, que inicia un *cluster* de tipo *FORK* con `numero_nodos` nodos (la cantidad de *workers* para el proceso a paralelizar).

Este tipo de *cluster* no está disponible en Windows, con lo que se debe usar un cluster de tipo *PSOCK* (Parallel Socket Cluster), con `type = PSOCK`. La diferencia principal reside en que con *FORK* todas las variables y funciones quedan automáticamente exportadas al *cluster*, con lo que no tenemos que preocuparnos de hacerlo manualmente.

Vamos a ilustrar esto con un **ejemplo**. Además de la librería `parallel`, también vamos a cargar la librería `microbenchmark`, para hacer pruebas de rendimiento:

In [ ]:
library(parallel) 
library(microbenchmark)

La función `detectCores()` permite ver la cantidad de núcleos/nodos disponibles en el sistema:

In [ ]:
detectCores()

[1] 8

Lo ideal es siempre dejar libre al menos uno para el sistema, de modo que la práctica más habitual es utilizar como número de nodos `detectCores()-1`, 

In [ ]:
cl <- makeCluster(spec = detectCores()-1, type = "FORK") # iniciamos el cluster
cl

socket cluster with 7 nodes on host ‘localhost’

Vamos a crear una lista compuesta por matrices grandes y utilizamos paralelización para calcular el valor medio de cada matriz:

In [ ]:
A.list <- replicate(replicate(expr = sample(1:50, size = 50, replace = T), n = 30), n = 20, simplify = F) # lista con 20 elementos, cada uno de ellos una matriz 50x30
str(A.list)

List of 20
 $ : int [1:50, 1:30] 43 17 1 26 45 33 19 26 23 4 ...
 $ : int [1:50, 1:30] 35 6 6 42 35 13 11 20 18 15 ...
 $ : int [1:50, 1:30] 27 14 10 43 19 46 1 19 40 14 ...
 $ : int [1:50, 1:30] 20 28 19 1 36 24 16 32 4 18 ...
 $ : int [1:50, 1:30] 7 41 25 24 15 4 14 23 23 40 ...
 $ : int [1:50, 1:30] 20 46 43 41 28 38 26 29 31 32 ...
 $ : int [1:50, 1:30] 34 8 37 15 4 19 24 28 30 27 ...
 $ : int [1:50, 1:30] 13 33 37 46 27 18 11 34 22 16 ...
 $ : int [1:50, 1:30] 18 28 44 27 24 32 40 25 10 30 ...
 $ : int [1:50, 1:30] 47 8 33 12 3 33 50 49 3 46 ...
 $ : int [1:50, 1:30] 3 27 18 4 31 27 36 31 7 27 ...
 $ : int [1:50, 1:30] 35 3 19 36 24 48 28 21 37 1 ...
 $ : int [1:50, 1:30] 44 43 33 17 43 20 50 8 28 10 ...
 $ : int [1:50, 1:30] 44 25 49 15 4 15 18 47 33 27 ...
 $ : int [1:50, 1:30] 10 25 19 27 5 43 11 32 15 9 ...
 $ : int [1:50, 1:30] 17 25 49 11 18 6 4 39 39 9 ...
 $ : int [1:50, 1:30] 38 31 26 17 22 40 29 42 16 37 ...
 $ : int [1:50, 1:30] 39 50 13 36 50 7 2 47 13 23 ...
 $ : int 

In [ ]:
cmean <- parLapply(cl = cl, X = A.list,  mean) # media de cada matriz
# cmean <- lapply(X = A.list,  mean) # media de cada matriz
print(cmean)

Conviene tener en cuenta que no todo es ideal paralelizarlo. De hecho, la operación que acabamos de hacer no tiene sentido a nivel de rendimiento. Comprobemos el rendimiento de `parApply()` contra `apply()`:

In [ ]:
#?microbenchmark
mbp <- microbenchmark(parLapply(cl = cl, X = A.list,  mean), times = 10)
mb <- microbenchmark(lapply(X = A.list,  mean), times = 10)

In [ ]:
mean(mbp$time)/10^9 # segundos

[1] 0.03995932

In [ ]:
mean(mb$time)/10^9 # segundos

[1] 0.0001057141

Siempre que acabemos de paralelizar hay que detener el cluster que creamos, utilizando `stopCluster()`:

In [ ]:
stopCluster(cl)

**Nota**: La función `parMapply()` no existe, por lo que si se quiere paralelizar un `mapply()` se debe acudir a `mcmapply()`. También existe `mclapply()`, que funciona análogamente a `lapply()`. En éstas no hay que iniciar el *cluster* de antemano, pero no están disponibles en Windows.
 
 Usando `mclapply()` paralelizar es tan sencillo como:

In [ ]:
mclapply(X = A.list, FUN = mean, mc.cores = 19)

[[1]]
[1] 25.05667

[[2]]
[1] 25.83

[[3]]
[1] 24.922

[[4]]
[1] 25.60867

[[5]]
[1] 25.56667

[[6]]
[1] 25.62933

[[7]]
[1] 25.79267

[[8]]
[1] 24.79267

[[9]]
[1] 25.51867

[[10]]
[1] 24.892

[[11]]
[1] 25.984

[[12]]
[1] 24.97067

[[13]]
[1] 25.01667

[[14]]
[1] 25.22867

[[15]]
[1] 25.24933

[[16]]
[1] 25.958

[[17]]
[1] 25.446

[[18]]
[1] 24.966

[[19]]
[1] 25.516

[[20]]
[1] 25.79733

Nótese que `mclapply()` abre y cierra el *cluster* automáticamente.

### ¿Cuándo paralelizar?

Aunque en teoría cada núcleo/nodo/procesador debería reducir linealmente el tiempo de cómputo, en la práctica hay trabajo adicional que reduce la eficiencia. Tanto el código como los datos necesitan ser transferidos a cada núcleo y se deben crear los subprocesos, lo que suma tiempo. Es por ello que vemos que paralelizar en el ejemplo de arriba no merece la pena, ya que si el tiempo de computación es muy corto para cada subtarea, el coste de preparar los recursos para la paralelización es más grande que el ahorro conseguido. 

A continuación vamos a ver un ejemplo de un caso en el que la paralelización disminuye sustancialmente el tiempo de computación. Tenemos una lista de $20$ vectores de tamaño $500$ con números obtenidos de una distribución uniforme, `unif`, al que queremos aplicar la función `mi.funcion()`, que definimos a continuación:

In [ ]:
unif <- replicate(runif(n = 500), n = 20, simplify = F)  # lista compuesta por 20 vectores, cada uno de ellos con 500 elementos

# extrae muestras de 10 elementos de cada vector hasta que obtiene una con la media y desviación típica razonablemente cerca
mi.funcion <- function(x) {
    sort(x)
    d.tipica <- sd(x)
    media <- mean(x)
    
    error.rel.media <- 1
    error.rel.d.tipica <- 1
    while(error.rel.media > 0.01 || error.rel.d.tipica > 0.01){
        muestra <- sample(x, size = 10)
        error.rel.media <- abs(mean(muestra) - media)/media
        error.rel.d.tipica <- abs(sd(muestra) - d.tipica)/d.tipica
    }
    return(muestra)
}

**Ejercicio:** Utiliza la función `microbenchmark()` para medir el tiempo de ejecución de aplicar la función `mi.funcion` a cada elemento de la lista `unif`, primero sin paralelizar y después paralelizando (realiza $50$ ejecuciones).

In [ ]:
# sin paralelizar


In [ ]:
# paralelizando


In [ ]:
# resultado: comparación de tiempos